<a href="https://colab.research.google.com/github/nonamesims4/The-Sims-4-MOD-JSON-/blob/main/sims4json%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E7%BF%BB%E8%A8%B3%E4%BF%AE%E6%AD%A3%E3%83%84%E3%83%BC%E3%83%AB%E5%85%B1%E6%9C%89%E7%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**実行時の警告についてのご案内**

このColabノートブックはGitHubなどGoogle以外のソースから読み込まれているため、Colab上で実行時に以下のような警告メッセージが表示されることがあります。

*「警告: このノートブックは Google が作成したものではありません。悪意のあるコードが含まれている可能性もあります。実行前にソースコードの確認を推奨します。」*

この警告はGoogle側からの標準的な注意喚起であり、ノートブックの内容や安全性を保証するものではありません。
本ツールはThe Sims 4 MODのJSON翻訳処理を目的としており、Google Drive内の個人情報アクセスや外部不正通信は行いません。

安心してご利用いただくために、コードの中身を目を通して理解したうえで実行することをおすすめします。

不安がある場合は、こちらのGitHubリポジトリのページもご確認ください。
https://github.com/nonamesims4/The-Sims-4-MOD-JSON-


In [ ]:
# Colab用: The Sims 4 MOD用JSON一括日本語化スクリプト
# 日本語のみの文章はスキップするがタグ置換＆s4 stbl mergeの翻訳語の先頭アスタリスク削除は必ず行う
# 英語と日本語混在のみ翻訳実施。プレースホルダー保護＆強化復元済み。
# 1セル完結・丁寧なコメント付き

# deep-translator インストール（初回のみ）
!pip install deep-translator

from google.colab import files
import json
import re
from deep_translator import GoogleTranslator
import time
from IPython.display import FileLink, display

# JSONファイルアップロード（完了で処理開始）
uploaded = files.upload()
input_path = next(iter(uploaded))

# プレースホルダーを安全なトークンに置換（タグ保護）
def mask_placeholders(text):
    # {}内に英数字・アンダースコア・ドット・アポストロフィ・ハイフンを許可
    pattern = r'\{[0-9a-zA-Z\._\'\-]+\}'
    found = re.findall(pattern, text)
    masked = text
    placeholder_map = {}
    for i, ph in enumerate(found):
        token = f'__PH_{i}__'   # スペースなし統一トークン
        masked = masked.replace(ph, token, 1)
        placeholder_map[token] = ph
    return masked, placeholder_map

# トークンの分割や空白混入でも復元可能なアンマスク関数
def unmask_placeholders(text, placeholder_map):
    for token, ph in placeholder_map.items():
        # 各文字の間に任意の空白が入る可能性に対応
        escaped_chars = list(re.escape(c) for c in token)
        pattern_str = r'\s*'.join(escaped_chars)
        pattern = re.compile(pattern_str, re.MULTILINE)
        text = pattern.sub(ph, text)
    return text

# タグの英語→日本語置換辞書
# 小文字は大文字に正規化してからこの辞書を適用
replace_dict = {
    '{M0.HE}': '{M0.彼}',
    '{M0.HIS}': '{M0.彼}',
    "{M0.HE'S}": "{M0.彼}",
    '{F0.SHE}': '{F0.彼女}',
    '{F0.HER}': '{F0.彼女}',
    "{F0.SHE'S}": "{F0.彼女}",
    '{M1.HE}': '{M1.彼}',
    '{M1.HIS}': '{M1.彼}',
    '{F1.SHE}': '{F1.彼女}',
    '{F1.HER}': '{F1.彼女}',

    '{0.SIMFIRSTNAME}': '{0.SimFirstName}',
    '{1.SIMFIRSTNAME}': '{1.SimFirstName}',

    '{0.SIMPRONOUNSUBJECTIVE}': '{M0.彼}{F0.彼女}',
    '{0.SIMPRONOUNPOSSESSIVEDEPENDENT}': '{M0.彼}{F0.彼女}',
    '{0.SIMPRONOUNREFLEXIVE}': '{M0.彼}{F0.彼女}',
    '{0.SIMPRONOUNOBJECTIVE}': '{M0.彼}{F0.彼女}',

    # 連続する同じタグの除去
    '{F0.彼女の}{F0.彼女}': '{F0.彼女}',
    '{F0.彼女}{F0.彼女の}': '{F0.彼女}',
    '{M0.彼の}{M0.彼}': '{M0.彼}',
    '{M0.彼}{M0.彼の}': '{M0.彼}',

    # 日本語の助詞などの除去
    '{F0.彼女の}の': '{F0.彼女}',
    '{M0.彼の}の': '{M0.彼}',
    '{F0.彼女に}に': '{F0.彼女}',
    '{M0.彼に}に': '{M0.彼}',
    '{F0.彼女が}が': '{F0.彼女}',
    '{M0.彼が}が': '{M0.彼}',
    '{F0.彼女は}は': '{F0.彼女}',
    '{M0.彼は}は': '{M0.彼}',
    '{F0.彼女を}を': '{F0.彼女}',
    '{M0.彼を}を': '{M0.彼}',
    '{F0.彼女と}と': '{F0.彼女}',
    '{M0.彼と}と': '{M0.彼}',
    '{F0.彼女の}{F0.彼女}は': '{F0.彼女}',
    '{M0.彼の}{M0.彼}は': '{M0.彼}',
}

# タグ置換関数
def normalize_tags(text):
    # まずタグ内の英字を大文字に正規化
    def upper_case_tags(match):
        return match.group(0).upper()
    text = re.sub(r'\{[0-9a-zA-Z\._\'\-]+\}', upper_case_tags, text)

    # 辞書による置換
    for k, v in replace_dict.items():
        text = text.replace(k, v)
    return text

# 日本語判定（ひらがな・カタカナ・漢字の有無）
def is_japanese(text):
    if not text:
        return False
    return bool(re.search(r'[\u3040-\u30ff\u4e00-\u9fff]', text))

# 行頭のアスタリスク(*)を全行から削除
def remove_all_leading_asterisks(text):
    lines = text.splitlines()
    cleaned_lines = [re.sub(r'^\s*\*\s*', '', line) for line in lines]
    return '\n'.join(cleaned_lines)

# 安全に翻訳（deep-translator利用、リトライ付き）
def safe_translate(text, max_retries=3):
    if not text.strip():
        return text
    for i in range(max_retries):
        try:
            translated = GoogleTranslator(source='en', target='ja').translate(text)
            if translated:
                return translated
        except Exception as e:
            print(f"翻訳エラー（試行 {i+1}）：{e}")
            time.sleep(2)
    print(f"翻訳失敗: {text[:50]}...")
    return text

# 長文を文単位に分割して翻訳（翻訳API制限対策）
def translate_long_text(text, maxlen=4500):
    if len(text) <= maxlen:
        return safe_translate(text)
    sentences = re.split(r'(?<=[.!?。！？\n])', text)
    out = ''
    buf = ''
    for s in sentences:
        if len(buf) + len(s) > maxlen:
            out += safe_translate(buf)
            buf = s
        else:
            buf += s
    if buf:
        out += safe_translate(buf)
    return out

print('JSONファイルを読み込み中・・・')
with open(input_path, encoding='utf-8') as f:
    data = json.load(f)

translated_count = 0
skipped_count = 0

for idx, entry in enumerate(data):
    val = entry.get('value', '')
    if not val:
        continue

    # まずタグの正規化（小文字→大文字、辞書置換）
    normalized_val = normalize_tags(val)

    if is_japanese(normalized_val):
        # 日本語のみなら翻訳しないが、先頭の*は削除
        cleaned = remove_all_leading_asterisks(normalized_val)
        entry['value'] = cleaned
        skipped_count += 1
        continue

    # 英字混在は翻訳対象
    print(f'[{idx+1}/{len(data)}] 翻訳中: {normalized_val[:40]}')
    masked, placeholder_map = mask_placeholders(normalized_val)
    translated = translate_long_text(masked)
    restored = unmask_placeholders(translated, placeholder_map)
    fixed = remove_all_leading_asterisks(restored) # 翻訳後に再度先頭の*を削除
    entry['value'] = fixed

    translated_count += 1
    time.sleep(0.1)

print(f'翻訳完了: {translated_count}件、スキップ: {skipped_count}件')

output_path = "output_translated.json"
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print(f'翻訳結果を {output_path} に保存しました。')

# Colab自動ダウンロード（失敗時はリンク表示）
try:
    files.download(output_path)
except Exception as e:
    print("自動ダウンロードに失敗しました。以下のリンクからダウンロードしてください。")
    display(FileLink(output_path))

Saving _Japanese.json to _Japanese.json
JSONファイルを読み込み中・・・
[1/3005] 翻訳中: Fairy
[2/3005] 翻訳中: Fairies are magically talented beings wi
[3/3005] 翻訳中: FairyForm
[4/3005] 翻訳中: Magic Energy
[6/3005] 翻訳中: (From Fairy Magic)
[7/3005] 翻訳中: Fire
[8/3005] 翻訳中: Water
[9/3005] 翻訳中: Nature
[10/3005] 翻訳中: Air
[11/3005] 翻訳中: Music
[12/3005] 翻訳中: Technology
[13/3005] 翻訳中: Crystals
[14/3005] 翻訳中: The Sun
[15/3005] 翻訳中: The Moon
[16/3005] 翻訳中: Emotion
[17/3005] 翻訳中: Darkness
[18/3005] 翻訳中: Ice
[19/3005] 翻訳中: Light
[20/3005] 翻訳中: Getting There
[22/3005] 翻訳中: Good Progress
[24/3005] 翻訳中: Ready for Advancement
[26/3005] 翻訳中: Channel Fairy Magic
[27/3005] 翻訳中: Repress Fairy Magic
[28/3005] 翻訳中: Float
[29/3005] 翻訳中: Fly Here
[30/3005] 翻訳中: Witch
[31/3005] 翻訳中: Power Spar
[32/3005] 翻訳中: Fairy Magic
[33/3005] 翻訳中: All fairies need to practice and grow th
[34/3005] 翻訳中: Acquired Fairy Magic Skill
[36/3005] 翻訳中: Reached Fairy Magic Level 2
[38/3005] 翻訳中: Reached Fairy Magic Level 3
[40/3005] 翻訳中: Reached Fairy Ma